  #                                    ΑΠΑΛΛΑΚΤΙΚΗ ΕΡΓΑΣΙΑ

# Βιρβιδάκη Χρυσούλα 1115201600021

In [1]:
import pandas as pd 
import string
import re
import numpy as np
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import f1_score, accuracy_score
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from collections import defaultdict
from nltk import pos_tag, word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

# Διάβασμα από αρχεία δεδομένων και αποθήκευση της πληροφορίας

In [2]:
# Στο X_train αποθηκεύω την πληροφορία του κάθε σχολίου που ανήκει στο αρχείο εκπαίδευσης. (Μέγεθος (3947, 2))
# Στο y_train αποθηκεύω την κατηγορία στην οποία ανήκει το κάθε ένα σχόλιο από το αρχείο εκπαίδευσης. (Μέγεθος (3947, 1))
# Στο X_test αποθηκεύω την πληροφορία του κάθε σχολίου που ανήκει στο αρχείο δοκιμής. (Μέγεθος (2235, 2))
# Στο y_test αποθηκεύω την κατηγορία στην οποία ανήκει το κάθε ένα σχόλιο από το αρχείο δοκιμής. (Μέγεθος (2235, 1))

In [3]:
X_train = pd.read_csv("data/train.csv",usecols=['Date', 'Comment']) 
y_train = pd.read_csv("data/train.csv",usecols=['Insult'])


X_test = pd.read_csv("data/impermium_verification_set.csv",usecols=['Date', 'Comment']) 
y_test = pd.read_csv("data/impermium_verification_labels.csv",usecols=['Insult'])



# Προεπεξεργασία και καθάρισμα των δεδομένων

In [4]:
# Αρχικά φροντίζω να καθαρίσω το αρχείο από τους χαρακτήρες  “\n”, "\xa0", "\xc2", "\u", "\x".
# Στη συνέχεια καθαρίζω από τα urls.
# Έπειτα καθαρίζω από συμβολοσειρές που περιέχουν το σύμβολο '@', καθώς και από συμβολοσειρές που περιέχουν αριθμούς,
# αλλά και τις ημερομηνίες,καθώς δεν δίνουν κάποια χρήσιμη πληροφορία, αλλά το αντίθετο.
# Τέλος, αφαιρώ τα σημεία στίξης και μετατρέπω όλους τους χαρακτήρες σε μικρούς.

In [5]:
# Καθαρισμός των test δεδομένων
    
X_test['Comment'] = X_test['Comment'].astype(str)
X_test['Comment'] = X_test['Comment'].apply(lambda x: x.replace('\\n', ' '))
X_test['Comment'] = X_test['Comment'].apply(lambda x: x.replace('\\xa0', ' '))
X_test['Comment']=X_test['Comment'].replace(r'\\xc2', u' ',regex=True)
X_test['Comment']=X_test['Comment'].replace(r'\\u', u' ',regex=True)
X_test['Comment']=X_test['Comment'].replace(r'\\x', u' ',regex=True)

X_test['Comment'] = X_test['Comment'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

X_test['Comment'].replace(r"\S+\@\S+", " ", regex=True ,inplace=True)
X_test['Comment'].replace(r"\S+\d+\S+", " ", regex=True,inplace=True)
X_test['Comment'].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", " DATE ", regex=True,inplace=True)

table = str.maketrans(dict.fromkeys(string.punctuation))
X_test['Comment'] = X_test['Comment'].apply(lambda x: x.translate(table))
X_test['Comment'] = X_test['Comment'].apply(lambda x: x.lower())

In [6]:
# Καθαρισμός των train δεδομένων

X_train['Comment'] = X_train['Comment'].astype(str)
X_train['Comment'] = X_train['Comment'].apply(lambda x: x.replace('\\n', ' '))
X_train['Comment'] = X_train['Comment'].apply(lambda x: x.replace('\\xa0', ' '))
X_train['Comment']=X_train['Comment'].replace(r'\\xc2', u' ',regex=True)
X_train['Comment']=X_train['Comment'].replace(r'\\u', u' ',regex=True)
X_train['Comment']=X_train['Comment'].replace(r'\\x', u' ',regex=True)

X_train['Comment'] = X_train['Comment'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

X_train['Comment'].replace(r"\S+\@\S+", " ", regex=True ,inplace=True)
X_train['Comment'].replace(r"\S+\d+\S+", " ", regex=True,inplace=True)
X_train['Comment'].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", " DATE ", regex=True,inplace=True)

table = str.maketrans(dict.fromkeys(string.punctuation))
X_train['Comment'] = X_train['Comment'].apply(lambda x: x.translate(table))
X_train['Comment'] = X_train['Comment'].apply(lambda x: x.lower()) 



## Μετατροπή των σχολίων σε word vectors χρησιμοποιώντας τον CountVectorizer 

In [7]:
X_train_text = X_train["Comment"].values
y_train_text = y_train.values

X_test_text = X_test["Comment"].values
y_test_text = y_test.values

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_text)
X_test_counts = count_vect.transform(X_test_text)


In [8]:
# Δημιουργώ αντίγραφα των πινάκων γιατί στην συνέχεια θα τα χρειαστώ
new_X_train = X_train.copy()
new_X_test = X_test.copy()


# Classification  NaiveBayes 

In [9]:
gnb = GaussianNB()
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

# Μετρικές απόδοσης: classification accuracy, F1 score

In [10]:
# Αρχικά υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [11]:
f1s = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(),  scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), scoring='accuracy')
print("Train accuracy", np.mean(accuracy))


Train f1s 0.6693385645211432
Train accuracy 0.6572065264475141


In [12]:
# Έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης

In [13]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel())
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.575294441389866
Test Accuracy:  0.5901565995525727


# Βελτιστοποίηση του Naive bayes

## (1)  lemmatization

In [14]:
lemmatizer=WordNetLemmatizer()
new_X_train['Comment'] = X_train['Comment'].apply(lambda x: [ lemmatizer.lemmatize(w,"n") for w in nltk.word_tokenize(x)])
new_X_test['Comment'] = X_test['Comment'].apply(lambda x: [ lemmatizer.lemmatize(w,"n") for w in nltk.word_tokenize(x)])

In [15]:
# Έπειτα από το lemmatization μετατρέπω τις λίστες των λέξεων που αποτελούν ένα σχόλιο σε συμβολοσειρά, και ακολουθώ
# την ίδια διαδικασία με πριν, μετατρέποντας τα σχόλια σε word vectors

In [16]:
for i,j in new_X_train.iterrows(): 
    j["Comment"] = ' '.join(map(str, j["Comment"]))
for i,j in new_X_test.iterrows(): 
    j["Comment"] = ' '.join(map(str, j["Comment"]))
    
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_text)
X_test_counts = count_vect.transform(X_test_text)

In [17]:
# Εφαρμόζω και πάλι το Classification NaiveBayes

In [18]:
gnb = GaussianNB()
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

In [19]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [20]:
f1s = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.6648367875719192
Train accuracy 0.6506194349521104


In [21]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [22]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel())
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.5664088461738794
Test Accuracy:  0.5816554809843401


## (2) stop words

In [23]:
# Χρησιμοποιώ τα δεδομένα τα αρχικά και όχι αυτά που προέκυψαν από τις προηγούμενες βελτιστοποιήσεις.

In [24]:
new_X_train = X_train.copy()
new_X_test = X_test.copy()

In [25]:
stop_words = set(stopwords.words('english'))

new_X_train['Comment'] = [w for w in X_train['Comment'] if not w in stop_words]
new_X_test['Comment'] = [w for w in X_test['Comment'] if not w in stop_words]

In [26]:
# Έπειτα από την αφαίρεση των stop words ακολουθώ την ίδια διαδικασία με πριν, μετατρέποντας τα σχόλια σε word vectors

In [27]:
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(X_train_text)
X_test_counts = count_vect.transform(X_test_text)

In [28]:
# Εφαρμόζω και πάλι το Classification NaiveBayes

In [29]:
gnb = GaussianNB()
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

In [30]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [31]:
f1s = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(),scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.668140903228535
Train accuracy 0.6554337328135278


In [32]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [33]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel())
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.5723712016794856
Test Accuracy:  0.5883668903803132


## (3) bigrams 

In [34]:
# Χρησιμοποιώ τα δεδομένα τα αρχικά και όχι αυτά που προέκυψαν από τις προηγούμενες βελτιστοποιήσεις.

In [35]:
new_X_train = X_train.copy()
new_X_test = X_test.copy()

In [36]:
# Ακολουθώ την ίδια διαδικασία με πριν, μετατρέποντας τα σχόλια σε word vectors προσθέτωντας αυτή τη φορά την παράμετρο
# ngram_range = (2,2) με την οποία χρησιμοποιώ φράσεις 2 λέξεων, αντί για μοναδικές λέξεις

In [37]:
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = CountVectorizer(ngram_range = (2,2))
X_train_counts = count_vect.fit_transform(X_train_text)

X_test_counts = count_vect.transform(X_test_text)

In [38]:
# Εφαρμόζω και πάλι το Classification NaiveBayes

In [39]:
gnb = GaussianNB()
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

In [40]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [41]:
f1s = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), cv=10, scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), cv=10,scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.6889410450222749
Train accuracy 0.683310415729615


In [42]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [43]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel(), cv=9)
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.613814555920073
Test Accuracy:  0.6214765100671141


## (4) Laplace Smoothing 

In [44]:
# Χρησιμοποιώ τα δεδομένα τα αρχικά και όχι αυτά που προέκυψαν από τις προηγούμενες βελτιστοποιήσεις.

In [45]:
new_X_train = X_train.copy()
new_X_test = X_test.copy()

In [46]:
# Ακολουθώ την ίδια διαδικασία με πριν, μετατρέποντας τα σχόλια σε word vectors

In [47]:
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = CountVectorizer(ngram_range = (1,2))
X_train_counts = count_vect.fit_transform(X_train_text)

X_test_counts = count_vect.transform(X_test_text)

In [48]:
# Εφαρμόζω και πάλι το Classification NaiveBayes. Αυτή τη φορά χρησιμοποιώ τον MultinomialNB ο οποίος μου δίνει την δυνατότητα
# να προσθέσω την παράμετρο alpha δίνοντας της τιμή 1.0, ώστε να κάνω την βελτιστοποίηση με Laplace Smoothing

In [49]:
gnb = MultinomialNB(alpha=1.0)
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

In [50]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [51]:
f1s = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), cv=10,scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(simple_gnb, X_train_counts.toarray(), y_train_text.ravel(), cv=10,scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.7646584138944055
Train accuracy 0.7565205937158646


In [52]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [53]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel(), cv=10)
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.6400059706986009
Test Accuracy:  0.6568232662192394


# Συμπεράσματα

In [54]:
# Τα scores για τα test αρχεία πριν κάνω οποιαδήποτε βελτιστοποίηση κυμαίνονταν το f1 score στο 0.57 ενώ το Accuracy στο 0.59
# Παρατήρησα ότι με τους δύο πρώτους τρόπους βελτιστοποίησης δεν υπήρξε κάποια αλλαγή στα σκορ, παρέμειναν στις ίδιες περίπου τιμές
# Αντίθετα με την χρήση των bigrams παρατήρησα μία αύξηση στα scores καθώς οι τιμές τους ανέβηκαν στο 0.61 και 0.62 αντίστοιχα.
# Ακόμα μεγαλύτερη αύξηση υπήρξε με τη χρήση του Laplace Smoothing, όπου τα scores ανέβηκαν στις τιμές 0.64 και 0.65 αντίστοιχα.
# Καταλήγουμε λοιπόν στο ότι για τα συγκεκριμένα δεδομένα μας πιο αποδοτικές μέθοδοι βελτιστοποίησης ήταν η χρήση των bigrams
# και του Laplace Smoothing.

# Δημιουργία πιο σύνθετου πίνακα χαρακτηριστικών με part-of-speech χαρακτηριστικά και TF-IDF-based χαρακτηριστικά

In [55]:
# Ακολουθώ την ίδια διαδικασία με πριν, μετατρέποντας τα σχόλια σε word vectors, με τη διαφορά ότι τώρα χρησιμοποιώ τον
# TfidfVectorizer

In [56]:
new_X_train = X_train.copy()
new_X_test = X_test.copy()

In [57]:
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_features=200,min_df=1)  #len=200
X_train_counts = count_vect.fit_transform(X_train_text)     # shape (3947,200)
X_test_counts = count_vect.transform(X_test_text)           # shape (2235,200)


In [58]:
# Παρακάτω για κάθε ένα σχόλιο βρίσκω τα part-of-speech χαρακτηριστικά των λέξεων του. Μετράω το σύνολο των λέξεων του σχολίου
# που είναι noun, verb, adverb και adjective και στη συνέχεια υπολογίζω τα ποσοστά αυτών με βάση τον συνολικό αριθμό των λέξεων
# του σχολίου. Αυτά τα ποσοστά τα αποθηκεύω σε ένα Dataframe με 4 στήλες (1 για το κάθε ποσοστό). Αυτό το Dataframe στο τέλος
# της επανάληψης θα έχει τόσες γραμμές όσα και τα σχόλια του κάθε αρχείου.

In [59]:
# Αρχικά δουλεύω πάνω στα train δεδομένα

POS_train = pd.DataFrame(columns = ['fractionAdverbs', 'fractionVerbs', 'fractionAdjectives', 'fractionNouns']) 

for i,j in X_train.iterrows(): 
    
    tokens = nltk.word_tokenize(j["Comment"])
    tags = nltk.pos_tag(tokens)
    counts = Counter( tag for word,  tag in tags)
    if len(tags)!=0:
        nouns = (counts['NN']+counts['NNS']+counts['NNP']+counts['NNPS'])/len(tags)
        verbs = (counts['VB']+counts['VBG']+counts['VBD']+counts['VBN']+counts['VBP']+counts['VBZ'])/len(tags)
        adverbs = (counts['RB']+counts['RBR']+counts['RBS'])/len(tags)
        adjectives = (counts['JJ']+counts['JJR']+counts['JJS'])/len(tags)
        
        POS_train = POS_train.append({'fractionAdverbs' : adverbs , 'fractionVerbs' : verbs,'fractionAdjectives' : adjectives,'fractionNouns' : nouns} , ignore_index=True)
    else:
        POS_train = POS_train.append({'fractionAdverbs' : 0 , 'fractionVerbs' : 0,'fractionAdjectives' : 0,'fractionNouns' : 0} , ignore_index=True)

In [60]:
# Αφού έχω υπολογίσει τόσο τα TF-IDF-based χαρακτηριστικά όσο και τα  part-of-speech χαρακτηριστικά του κάθε σχολίου,
# τα ενώνω σε έναν ενιαίο πίνακα.

In [61]:
tfidf_pos_X_train = np.hstack((X_train_counts.toarray(),POS_train))

In [62]:
# Ακολουθώ την ίδια διαδικασία και για τα test δεδομένα

POS_test = pd.DataFrame(columns = ['fractionAdverbs', 'fractionVerbs', 'fractionAdjectives', 'fractionNouns']) 

for i,j in X_test.iterrows(): 
    
    tokens = nltk.word_tokenize(j["Comment"])
    tags = nltk.pos_tag(tokens)
    counts = Counter( tag for word,  tag in tags)
    if len(tags)!=0:
        nouns = (counts['NN']+counts['NNS']+counts['NNP']+counts['NNPS'])/len(tags)
        verbs = (counts['VB']+counts['VBG']+counts['VBD']+counts['VBN']+counts['VBP']+counts['VBZ'])/len(tags)
        adverbs = (counts['RB']+counts['RBR']+counts['RBS'])/len(tags)
        adjectives = (counts['JJ']+counts['JJR']+counts['JJS'])/len(tags)
        
        POS_test = POS_test.append({'fractionAdverbs' : adverbs , 'fractionVerbs' : verbs,'fractionAdjectives' : adjectives,'fractionNouns' : nouns} , ignore_index=True)
    else:
        POS_test = POS_test.append({'fractionAdverbs' : 0 , 'fractionVerbs' : 0,'fractionAdjectives' : 0,'fractionNouns' : 0} , ignore_index=True)

In [63]:
tfidf_pos_X_test = np.hstack((X_test_counts.toarray(),POS_test))

# Classification: SVM

In [64]:
clf = svm.SVC()
tfidf_clf = clf.fit(tfidf_pos_X_train, y_train_text.ravel())

In [65]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [66]:
f1s = cross_val_score(tfidf_clf, tfidf_pos_X_train, y_train_text.ravel(), scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(tfidf_clf, tfidf_pos_X_train, y_train_text.ravel(), scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.7843145830812822
Train accuracy 0.8084622419021033


In [67]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [68]:
y_pred = cross_val_predict(tfidf_clf, tfidf_pos_X_test, y_test_text.ravel())
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.6661764706585737
Test Accuracy:  0.6702460850111857


# Classification: Random Forests

In [69]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
tfidf_clf = clf.fit(tfidf_pos_X_train, y_train_text.ravel())

In [70]:
# Υπολογίζω τα scores με τις μετρικές αυτές για τα train δεδομένα

In [71]:
f1s = cross_val_score(tfidf_clf, tfidf_pos_X_train, y_train_text.ravel(), cv=10, scoring='f1_weighted')
print ('Train f1s', np.mean(f1s))

accuracy = cross_val_score(tfidf_clf, tfidf_pos_X_train, y_train_text.ravel(), cv=10,scoring='accuracy')
print("Train accuracy", np.mean(accuracy))

Train f1s 0.6217078924673287
Train accuracy 0.7342286191608302


In [72]:
# Και έπειτα υπολογίζω τα αντίστοιχα scores με τις ίδες μετρικές για τα test δεδομένα
# και αξιολογώ την απόδοση της μεθόδου κατηγοριοποίησης έπειτα από την βελτιστοποίηση

In [73]:
y_pred = cross_val_predict(tfidf_clf, tfidf_pos_X_test, y_test_text.ravel(), cv=10)
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.5800385986640617
Test Accuracy:  0.6308724832214765


# Συμπεράσματα

In [74]:
# Έπειτα από τα παραπάνω καταλήγω ότι ο συνδυασμός των part-of-speech χαρακτηριστικών και TF-IDF-based χαρακτηριστικών
# λειτούργησε πιο αποδιτκά με τον SVM Classifier. 
# Έπειτα από πειραματισμούς για την τιμή του ορίσματος ngram_range στον TfidfVectorizer κατέληξα ότι μεγαλύτερα σκορ
# έπαιρνα για ngram_range=(1,2).
# Ακόμα επειδή τα part-of-speech χαρακτηριστικά είναι μόνο 4 ανά σχόλιο, έπρεπε να ρυθμίσω τον αριθμό των features
# που δημιουργούνταν από τον TfidfVectorizer. Με πολλά πειράματα είδα ότι για max_features=200 παίρνω τα υψηλότερα σκορ.

# Πειραματισμοί για περαιτέρω βελτιστοποιήσεις

In [75]:
new_X_train = X_train.copy()
new_X_test = X_test.copy()

In [76]:
X_train_text = new_X_train["Comment"].values
y_train_text = y_train.values

X_test_text = new_X_test["Comment"].values
y_test_text = y_test.values

count_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_features=2000)
X_train_counts = count_vect.fit_transform(X_train_text)
X_test_counts = count_vect.transform(X_test_text)

In [77]:
#gnb = RandomForestClassifier(max_depth=15, random_state=0,n_estimators=50, max_features=1000) #0.65
#gnb = MultinomialNB(alpha=1.0) #0.70

gnb = svm.SVC(kernel='linear', C=1.0,gamma='scale') #0.71
simple_gnb = gnb.fit(X_train_counts.toarray(), y_train_text.ravel())

In [78]:
y_pred = cross_val_predict(simple_gnb, X_test_counts.toarray(), y_test_text.ravel())
F1_score = f1_score(y_test_text.ravel(), y_pred, average='weighted')
print('Test f1 score: ' , F1_score)

Accuracy = accuracy_score(y_test_text.ravel(), y_pred)
print("Test Accuracy: ", Accuracy)

Test f1 score:  0.7111200599173014
Test Accuracy:  0.7127516778523489


# Συμπεράσματα

In [79]:
# Αρχικά πειραματίστηκα με την χρήση το απλού CountVectorizer και με τις παραμέτρους του, όμως στο τέλος κατέληξα ότι υψηλότερα
# σκορ έπαιρνα με τον TfidfVectorizer. Ακόμα μεγάλες διαφορές είχε ο TfidfVectorizer ανάλογα με την τιμή που έδινα στο όρισμα:
# ngram_range με υψηλότερο τελικά σκορ να δίνει όταν χρησιμοποιώ τιμ΄΄΄η: (1,1).

# Ακόμα δοκίμασα διάφορους classifiers. Ανάμεσα σε αυτούς της οικογένειας Naive Bayes, καταλληλότερος ήταν με διαφορά ο 
# MultinomialNB(70%). Ωστόσο παρόμοια σκορ δίνει και ο SVM (πάνω από 70%). Αντίθετα ο KNeighborsClassifier στην περίπτωση μου
# δεν έδειξε να είναι πολύ αποδοτικός αφού τα σκορ που πήρα ήταν κάτω από 60%.
# Αρκετά αποδοτικός ήταν και ο RandomForestClassifier που έδωσε σκορ γύρω στο 65%.